## (1) 대출건수가 0건인 책의 권 수

In [3]:
import pandas as pd
lib = pd.read_csv('2023_01.csv')
lib1 = pd.read_csv('2023_01.csv')
lib2 = pd.read_csv('2023_02.csv')
lib3 = pd.read_csv('2023_03.csv')
lib4 = pd.read_csv('2023_04.csv')
lib5 = pd.read_csv('2023_05.csv')
lib6 = pd.read_csv('2023_06.csv')
lib7 = pd.read_csv('2023_07.csv')
lib8 = pd.read_csv('2023_08.csv')
lib9 = pd.read_csv('2023_09.csv')
lib10 = pd.read_csv('2023_10.csv')
lib11 = pd.read_csv('2023_11.csv')
lib12 = pd.read_csv('2023_12.csv')

# 파일명을 기반으로 각각의 데이터프레임에 월 정보 추가
lib1['월'] = '01'
lib2['월'] = '02'
lib3['월'] = '03'
lib4['월'] = '04'
lib5['월'] = '05'
lib6['월'] = '06'
lib7['월'] = '07'
lib8['월'] = '08'
lib9['월'] = '09'
lib10['월'] = '10'
lib11['월'] = '11'
lib12['월'] = '12'

# 리스트에 모든 데이터프레임을 넣기
dataframes = [lib1, lib2, lib3, lib4, lib5, lib6, lib7, lib8, lib9, lib10, lib11, lib12]

# concat 함수를 사용하여 모든 데이터프레임을 하나로 결합
combined_df = pd.concat(dataframes, ignore_index=True)

# 주제분류번호를 문자열로 변환 후 100 단위로 잘라서 그룹화
combined_df['주제분류번호'] = combined_df['주제분류번호'].astype(str)
combined_df['주제분류그룹'] = combined_df['주제분류번호'].apply(lambda x: x.split('.')[0] if '.' in x else x)
combined_df['주제분류그룹'] = combined_df['주제분류그룹'].apply(lambda x: x[:1] + '00')

# 주제분류그룹별로 가장 대출이 많은 책 Top 10을 찾기 (책 제목 + 출판사 기준)
top10_books_by_category = {}

for group, group_df in combined_df.groupby('주제분류그룹'):
    top10_books = group_df.groupby(['도서명', '출판사'])['대출건수'].sum().sort_values(ascending=False).head(10)
    top10_books_by_category[group] = top10_books

# 책 제목과 출판사를 기준으로 그룹화하여 대출건수 합산
grouped_books = combined_df.groupby(['도서명', '출판사'])['대출건수'].sum()

# 대출건수가 0인 경우를 필터링하여 해당하는 행의 수를 계산
zero_loan_count = (grouped_books == 0).sum()

# 결과 출력
print(f"대출 건수가 0인 책의 권 수: {zero_loan_count}")

C:\Users\shs21\AppData\Local\Temp\ipykernel_13764\191679494.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  lib = pd.read_csv('2023_01.csv')
C:\Users\shs21\AppData\Local\Temp\ipykernel_13764\191679494.py:3: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  lib1 = pd.read_csv('2023_01.csv')
C:\Users\shs21\AppData\Local\Temp\ipykernel_13764\191679494.py:4: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  lib2 = pd.read_csv('2023_02.csv')
C:\Users\shs21\AppData\Local\Temp\ipykernel_13764\191679494.py:5: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  lib3 = pd.read_csv('2023_03.csv')
C:\Users\shs21\AppData\Local\Temp\ipykernel_13764\191679494.py:6: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  lib4 = pd.read_

대출 건수가 0인 책의 권 수: 52777


## (2) 발행년도와 대출건수의 관계 살펴보기

In [80]:
# 발행년도를 정수형으로 변환 (불가능한 경우 NaN으로 변환)
combined_df['발행년도'] = pd.to_numeric(combined_df['발행년도'], errors='coerce')

# 8자리 발행년도 처리: 앞 4자리만 발행년도로 취급
combined_df['발행년도'] = combined_df['발행년도'].apply(lambda x: int(str(int(x))[:4]) if pd.notna(x) and len(str(int(x))) == 8 else x)

# 발행년도가 정수이고, 3자리 이하가 아닌 경우만 필터링
filtered_df = combined_df[(combined_df['발행년도'] >= 1000) & (combined_df['발행년도'] < 10000)]

# 발행년도별로 그룹화하여 총 대출건수를 합산
loan_count_by_year = filtered_df.groupby('발행년도')['대출건수'].sum()

# 대출건수가 많은 순으로 정렬
loan_count_by_year_sorted = loan_count_by_year.sort_values(ascending=False)

# 출력하기
print(loan_count_by_year_sorted)

발행년도
2008.0    4466509.0
2010.0    4299500.0
2011.0    4296596.0
2009.0    4114291.0
2007.0    4085844.0
            ...    
1973.0         24.0
2024.0          1.0
1959.0          0.0
1953.0          0.0
1937.0          0.0
Name: 대출건수, Length: 67, dtype: float64


## (3) 대출건수가 0인 발행년도 출력

In [52]:
# 대출건수가 0인 발행년도 필터링
zero_loan_years = loan_count_by_year[loan_count_by_year == 0]

# 결과 출력
print(zero_loan_years)

발행년도
1937.0    0.0
1953.0    0.0
1959.0    0.0
Name: 대출건수, dtype: float64


## (4) 대출건수가 적은 순으로 발행년도 출력

In [86]:
# 발행년도별로 그룹화하여 총 대출건수를 합산
loan_count_by_year = filtered_df.groupby('발행년도')['대출건수'].sum()

# 대출건수가 적은 순으로 전체 데이터를 정렬
loan_count_by_year_sorted = loan_count_by_year.sort_values(ascending=True)

# 결과 출력
print(loan_count_by_year_sorted)

발행년도
1937.0          0.0
1953.0          0.0
1959.0          0.0
2024.0          1.0
1973.0         24.0
            ...    
2007.0    4085844.0
2009.0    4114291.0
2011.0    4296596.0
2010.0    4299500.0
2008.0    4466509.0
Name: 대출건수, Length: 67, dtype: float64


## (5) 발행년도와 도서권수의 관계 살펴보기

In [103]:
# 발행년도를 정수형으로 변환 (불가능한 경우 NaN으로 변환)
combined_df['발행년도'] = pd.to_numeric(combined_df['발행년도'], errors='coerce')

# 8자리 발행년도 처리: 앞 4자리만 발행년도로 취급
combined_df['발행년도'] = combined_df['발행년도'].apply(lambda x: int(str(int(x))[:4]) if pd.notna(x) and len(str(int(x))) == 8 else x)

# 발행년도가 정수이고, 3자리 이하가 아닌 경우만 필터링
filtered_df2 = combined_df[(combined_df['발행년도'] >= 1000) & (combined_df['발행년도'] < 10000)]

# 발행년도별로 그룹화하여 총 대출건수를 합산
loan_count_by_year2 = filtered_df2.groupby('발행년도')['도서권수'].sum()

# 대출건수가 적은 순으로 정렬
loan_count_by_year_sorted2 = loan_count_by_year2.sort_values(ascending=True)

# 출력하기
print(loan_count_by_year_sorted2)

발행년도
2024.0         2.0
1953.0        11.0
1961.0        11.0
1937.0        12.0
2106.0        12.0
            ...   
2012.0    249969.0
2008.0    252044.0
2003.0    255031.0
2011.0    257728.0
2001.0    275350.0
Name: 도서권수, Length: 67, dtype: float64


In [105]:
print(loan_count_by_year_sorted2.head(20))

발행년도
2024.0       2.0
1953.0      11.0
1961.0      11.0
1937.0      12.0
2106.0      12.0
2108.0      12.0
1959.0      13.0
1964.0      24.0
1968.0      24.0
1967.0      36.0
1969.0      36.0
1970.0      36.0
1950.0      48.0
1965.0      60.0
1972.0      70.0
1971.0      97.0
1973.0     175.0
1974.0     294.0
1977.0     629.0
1981.0    1669.0
Name: 도서권수, dtype: float64
